## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-25-02-32-25 +0000,bbc,Should smartphones be locked away at gigs and ...,https://www.bbc.com/news/articles/cvgqxljxndeo...
1,2026-01-25-02-27-16 +0000,wapo,Live updates: Fatal shooting in Minneapolis by...,https://www.washingtonpost.com/nation/2026/01/...
2,2026-01-25-02-27-16 +0000,wapo,Minneapolis council member Aisha Chughtai ran...,https://www.washingtonpost.com
3,2026-01-25-02-26-47 +0000,nyt,Trump Threatens Canada With Tariffs as Post-Da...,https://www.nytimes.com/2026/01/24/world/canad...
4,2026-01-25-02-24-07 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/us/minneapol...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2456/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,21
10,minneapolis,20
88,ice,17
9,shooting,15
93,china,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2026-01-24-22-55-08 +0000,nypost,Chilling new video reveals new angle of fed sh...,https://nypost.com/2026/01/24/us-news/chilling...,98
18,2026-01-25-01-50-20 +0000,nypost,Chaotic moment federal officer reached for gun...,https://nypost.com/2026/01/24/us-news/moment-f...,84
77,2026-01-24-20-11-38 +0000,nypost,Gavin Newsom launches vicious attack on presid...,https://nypost.com/2026/01/24/us-news/gavin-ne...,81
45,2026-01-24-23-17-59 +0000,nypost,"Who is Alex Jeffrey Pretti, the anti-ICE prote...",https://nypost.com/us-news/who-is-alex-jeffrey...,73
20,2026-01-25-01-41-59 +0000,latimes,Peaceful protest over Minneapolis shooting beg...,https://www.latimes.com/california/story/2026-...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,98,2026-01-24-22-55-08 +0000,nypost,Chilling new video reveals new angle of fed sh...,https://nypost.com/2026/01/24/us-news/chilling...
111,67,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...
73,45,2026-01-24-20-47-10 +0000,nypost,"Trilateral talks between US, Russia and Ukrain...",https://nypost.com/2026/01/24/world-news/trila...
101,29,2026-01-24-17-06-00 +0000,wsj,China’s Xi Places His Top General Under Invest...,https://www.wsj.com/world/china/chinas-xi-plac...
16,28,2026-01-25-01-52-00 +0000,wsj,Senate Democrats signaled they are willing to ...,https://www.wsj.com/politics/policy/senate-dem...
52,28,2026-01-24-22-31-04 +0000,nyt,Here’s What to Know if You Plan to Travel Duri...,https://www.nytimes.com/2026/01/22/travel/wint...
130,28,2026-01-24-12-48-31 +0000,nypost,Trump tells The Post he’s skipping the Super B...,https://nypost.com/2026/01/24/us-news/trump-te...
53,26,2026-01-24-22-30-27 +0000,nypost,Feds were arresting Ecuadorian illegal alien J...,https://nypost.com/2026/01/24/us-news/feds-wer...
71,25,2026-01-24-20-48-42 +0000,nypost,Officer ‘will lose finger’ after unhinged Minn...,https://nypost.com/2026/01/24/us-news/unhinged...
39,24,2026-01-24-23-50-58 +0000,nypost,‘Aggressive’ raccoon that charged at a group o...,https://nypost.com/2026/01/24/us-news/aggressi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
